In [185]:
import numpy as np
import matplotlib.pyplot as plt
import time
Seed=np.random.get_state()
%store -r 
np.random.set_state(seed_4stages6_t_7_76)

def wave_effect(alpha,delta):
    '''
    Function that represents effect of each wave plate in the SOP
    '''
    
    wave_plate_matrix=[[np.cos(alpha)**2+(np.sin(alpha)**2)*np.cos(delta), np.cos(alpha)*np.sin(alpha)*(1-np.cos(delta)),np.sin(alpha)*np.sin(delta)],
                  [np.cos(alpha)*np.sin(alpha)*(1-np.cos(delta)),np.cos(delta)*np.cos(alpha)**2+np.sin(alpha)**2,-np.cos(alpha)*np.sin(delta)],
                  [-np.sin(alpha)*np.sin(delta),np.cos(alpha)*np.sin(delta),np.cos(delta)]]

    return wave_plate_matrix 

def V_A_value(alpha,delta,V_o,V_pi,V_A_bias):
    
    ''' 
    V A calculation
    '''
    
    V_A=2*V_o*delta*np.sin(alpha)-V_pi*delta*np.cos(alpha)+V_A_bias

    return V_A
 
def V_C_value(alpha,delta,V_o,V_pi,V_C_bias):
    
    ''' 
    V C calculation
    '''
    V_C=2*V_o*delta*np.sin(alpha)+V_pi*delta*np.cos(alpha)+V_C_bias

    return V_C

def f_lab(alpha_array,delta_array,V_Ain_matrix,V_Cin_matrix,start_array,goal_array,first_weight,sec_weight,third_weight,extra_sum,n_stages):
    ''' 
    alpha_array - array with the values of alpha for the various stages of a PSO particle
    delta_array - array with the values of delta for the various stages of a PSO particle
    V_Ain_matrix - matrix of values of VA for the previous polarization state for each stage
    V_Cin_matrix - matrix of values of VC for the previous polarization state for each stage
    start_array - Input State of Polarization as a Stokes Vector
    goal_array - Goal State of Polarization as a Stokes Vector
    first_weight,sec_weight,third_weight - weights of the cost function
    extra_sum - 1 or 0 , 1 if we are comparing to more than one previous calculated SOP
    n_stages - number of stages of the EPC used
    '''
    V_a_sum_array=np.zeros(n_stages)
    V_c_sum_array=np.zeros(n_stages)

    
    V_pi_array=np.array([56.3,56,55.9,56.1,56.1,56,56.1,56])
    V_o_array=np.array([27.2,27,27,26.9,26.8,27.2,26.8,27.2])

    #V_A,bias e V_C,bias não têm efeito no processo de otimização 
    V_A_bias_array=np.array([-10.7,-9.3,-8.5,-10.9,-7.4,-7.6,-7.4,-7.6])
    V_C_bias_array=np.array([8.4,9.6,9.4,11.1,11.6,10.9,11.6,10.9])

    first_sum=0
    second_sum=0
    third_sum=0
    
    delta_array_rad=delta_array*np.pi*2
    
    M=wave_effect(alpha_array[n_stages-1],delta_array_rad[n_stages-1])
    i=n_stages-2
    while(i>=0): 
        M=np.dot(M,wave_effect(alpha_array[i],delta_array_rad[i]))
        i-=1
    
    start_array=np.reshape(start_array,(3,1))
    
    calc_array=np.dot(M,start_array)
    
    calc_array=np.reshape(calc_array,(1,3))
    calc_array=calc_array[0]
    
    for i in range(0,3):
        first_sum+=abs((calc_array[i]-goal_array[i]))*first_weight#mudar função de ajuste
        
    
    if(extra_sum):   
        
        # print("VA linhas",len(V_Ain_matrix),"VA colunas",len(V_Ain_matrix[0]))
        # print("VC linhas",len(V_Cin_matrix),"VC colunas",len(V_Cin_matrix[0]))
        # print("before")
        for j in range(0,len(V_Ain_matrix)):
            # print("after",len(V_Ain_matrix))
            
            for i in range(0,n_stages):
                
                second_sum+=(abs(V_A_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_A_bias_array[i])-V_Ain_matrix[j,i])/(sec_weight*len(V_Ain_matrix)))**2
                #posso tentar normalizar estas diferenças

        for j in range(0,len(V_Cin_matrix)):
            for i in range(0,n_stages):
                third_sum+=(abs(V_C_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_C_bias_array[i])-V_Cin_matrix[j,i])/(third_weight*len(V_Cin_matrix)))**2
                
    else:
        for i in range(0,n_stages):
            temp_calc=(abs((V_A_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_A_bias_array[i])-V_Ain_matrix[i]))/sec_weight)**2
            second_sum+=temp_calc
            V_a_sum_array[i]=temp_calc
        #posso tentar normalizar estas diferenças
    
        for i in range(0,n_stages):
            temp_calc=(abs((V_C_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_C_bias_array[i])-V_Cin_matrix[i]))/third_weight)**2
            third_sum+=temp_calc
            V_c_sum_array[i]=temp_calc
    # z=(first_sum/3)+(second_sum/n_stages)+(third_sum/n_stages)
    z=first_sum+second_sum+third_sum#+np.std(V_a_sum_array)+np.std(V_c_sum_array)
    #print(first_sum,'~~~',second_sum,'~~',third_sum,'~~>',z)
    return z


In [186]:
def index_array(lower_Q1_delta,lower_Q1_alpha,V_A_std,alpha_increment,delta_increment):
    index_alpha_array=[]
    for item in range(0,V_A_std[1,:].size):
        
        if(V_A_std[1,item] <= lower_Q1_alpha):
            index_alpha_array.append(item)
        
    index_delta_array=[]

    for item in range(0,V_A_std[0,:].size):
        
        if(V_A_std[0,item] <= lower_Q1_delta):
            index_delta_array.append(item)
            

    # print("index de delta abaixo do 1º quartil",index_delta_array)
    i=0   
    sequence_delta_array=[]
    while(i<(len(index_delta_array)-1)):

        if((index_delta_array[i]-index_delta_array[i+1])!=-1):
            
            sequence_delta_array.append(i)
            
        i+=1

    # print("index separadores de valores delta",sequence_delta_array)

    # print("index de alpha abaixo do 1º quartil",index_alpha_array)
    i=0   
    sequence_alpha_array=[]
    while(i<(len(index_alpha_array)-1)):

        if((index_alpha_array[i]-index_alpha_array[i+1])!=-1):
            
            sequence_alpha_array.append(i)
            
        i+=1
        
    # print("index separadores de valores alpha",sequence_alpha_array)
    
    start_delta_array=[]
    end_delta_array=[]
    if(sequence_delta_array==[]):
        start_delta_array.append(delta_increment*index_delta_array[0])
        end_delta_array.append(delta_increment*index_delta_array[-1])
    else:
        start=1
        
        for item in range(0,len(index_delta_array)):
            if(start==1):
                start_delta_array.append(delta_increment*index_delta_array[item])
                start=0
            if(item in sequence_delta_array):
                end_delta_array.append(delta_increment*index_delta_array[item])
                start=1

        end_delta_array.append(delta_increment*index_delta_array[-1])
        
    start_alpha_array=[]
    end_alpha_array=[]
    if(sequence_alpha_array==[]):
        start_alpha_array.append(alpha_increment*index_alpha_array[0])
        end_alpha_array.append(alpha_increment*index_alpha_array[-1])
    else:

        start=1
        
        for item in range(0,len(index_alpha_array)):
            if(start==1):
                start_alpha_array.append(alpha_increment*index_alpha_array[item])
                start=0
            if(item in sequence_alpha_array):
                end_alpha_array.append(alpha_increment*index_alpha_array[item])
                start=1

        end_alpha_array.append(alpha_increment*index_alpha_array[-1])

    return(index_delta_array,index_alpha_array,start_delta_array,end_delta_array,start_alpha_array,end_alpha_array)


In [187]:
def interval_selection(V_pi,V_0,V_a_bias,V_c_bias):
    #parâmetros modificáveis 
    size=50

    ###
    alpha_array=np.zeros(size)
    delta_array=np.zeros(size)

    alpha_increment=(2*np.pi)/(alpha_array.size-1)
    delta_increment=1/(delta_array.size-1)


    alpha=0
    i=0
    while i<alpha_array.size:
        alpha_array[i]=alpha
        alpha+=alpha_increment
        i+=1


    delta=0
    i=0
    while i<delta_array.size:
        delta_array[i]=delta
        delta+=delta_increment
        i+=1

    V_a=np.zeros((delta_array.size,alpha_array.size))

    i=0
    while i<delta_array.size:
        j=0
        while j<alpha_array.size:
            V_a[i,j]=2*V_0*delta_array[i]*np.sin(alpha_array[j])-V_pi*delta_array[i]*np.cos(alpha_array[j])+V_a_bias
            j+=1
        i+=1

    V_c=np.zeros((delta_array.size,alpha_array.size))
    i=0
    while i<delta_array.size:
        j=0
        while j<alpha_array.size:
            V_c[i,j]=2*V_0*delta_array[i]*np.sin(alpha_array[j])+V_pi*delta_array[i]*np.cos(alpha_array[j])+V_c_bias
            j+=1
        i+=1
    
    V_C_std=np.zeros((2,size))
    V_A_std=np.zeros((2,size))

    ''' 
    A linha 1 apresenta as standard deviation das colunas de delta
    A linha 2 apresenta as standard deviation das linhas de alpha
    '''

    j=0
    while(j<2):
        i=0
        if(j==0):
            while(i<size):
                V_C_std[j,i]=np.std(V_c[i,:])
                V_A_std[j,i]=np.std(V_a[i,:])
                i+=1
        else:
            while(i<size):
                V_C_std[j,i]=np.std(V_c[:,i])
                V_A_std[j,i]=np.std(V_a[:,i])
                i+=1
        j+=1


    lower_Q1_delta=np.quantile(V_A_std[0,:],0.15)
    lower_Q1_alpha=np.quantile(V_A_std[1,:],0.15)

    # print("1º quantil distribuição de alpha para um delta para V_A",np.quantile(V_A_std[0,:],0.15))
    # print("1º quantil distribuição de alpha para um delta para V_C",np.quantile(V_C_std[0,:],0.15))
    # print("1º quantil distribuição de delta para um alpha para V_A",np.quantile(V_A_std[1,:],0.15))
    # print("1º quantil distribuição de delta para um alpha para V_C",np.quantile(V_C_std[1,:],0.15))

    index_delta_array_A,index_alpha_array_A,start_delta_array_A,end_delta_array_A,start_alpha_array_A,end_alpha_array_A=index_array(lower_Q1_delta,lower_Q1_alpha,V_A_std,alpha_increment,delta_increment)
    index_delta_array_C,index_alpha_array_C,start_delta_array_C,end_delta_array_C,start_alpha_array_C,end_alpha_array_C=index_array(lower_Q1_delta,lower_Q1_alpha,V_C_std,alpha_increment,delta_increment)

    V_C_delta_mean=V_c[index_delta_array_C,:]
    V_A_delta_mean=V_a[index_delta_array_A,:]
    V_C_alpha_mean=V_c[:,index_alpha_array_C]
    V_A_alpha_mean=V_a[:,index_alpha_array_A]

    V_A_mean=(np.sum(V_A_delta_mean)+np.sum(V_A_alpha_mean))/(np.size(V_A_delta_mean[0])*np.size(V_A_delta_mean[:,0])+(np.size(V_A_alpha_mean[0])*np.size(V_A_alpha_mean[:,0])))

    V_C_mean=(np.sum(V_C_delta_mean)+np.sum(V_C_alpha_mean))/(np.size(V_C_delta_mean[0])*np.size(V_C_delta_mean[:,0])+(np.size(V_C_alpha_mean[0])*np.size(V_C_alpha_mean[:,0])))
    
    # print("V_A_mean",V_A_mean)
    # print("V_C_mean",V_C_mean)
    
    return(start_delta_array_A,start_delta_array_C,end_delta_array_A,end_delta_array_C,start_alpha_array_A,start_alpha_array_C,end_alpha_array_A,end_alpha_array_C,V_A_mean,V_C_mean)

In [188]:
def particle_distribution(n_particles,start_delta_array,end_delta_array,start_alpha_array,end_alpha_array):
    
    val_delta=1
    val_alpha=1
    j=0
    counter=0
    j_delta=n_particles/len(start_delta_array)
    # print(j_delta)
    while(j<n_particles):
        
        if(val_delta==1):
            X_delta=np.random.rand(1, int(j_delta))*(end_delta_array[0]-start_delta_array[0])+start_delta_array[0]
            X_delta=X_delta[0]
            val_delta=0
    
        else:
            X_delta_add=np.random.rand(1, int(j_delta))*(end_delta_array[counter]-start_delta_array[counter])+start_delta_array[counter]
            X_delta_add=X_delta_add[0]                                                                                                              
            X_delta = np.concatenate((X_delta, X_delta_add))
        # print(counter)
        counter+=1
        j+=j_delta

    # print(start_alpha_array)
    
    l=0
    counter=0
    l_delta=n_particles/len(start_alpha_array)
    while(l<n_particles):
        # print(counter)
        if(val_alpha==1):
            X_alpha=np.random.rand(1, int(l_delta))*(end_alpha_array[0]-start_alpha_array[0])+end_alpha_array[0]
            X_alpha=X_alpha[0]
            val_alpha=0
        else:
            X_alpha_add=np.random.rand(1, int(l_delta))*(end_alpha_array[counter]-start_alpha_array[counter])+start_alpha_array[counter]
            X_alpha_add=X_alpha_add[0]
            X_alpha = np.concatenate((X_alpha, X_alpha_add))
            #tenho de mudar
            
        counter+=1
        l+=l_delta
    
    return(X_delta,X_alpha)

In [189]:
def PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,start_array,iteration_max,first_weight,sec_weight,third_weight,extra_sum,n_stages):
    #como as partículas ainda não começaram a sua melhor posição é a atual
    pbest=X
    #aplicamos os valores das coordadenadas de cada partícula à função
    pbest_obj=np.zeros(n_particles)

    for i in range(0,n_particles):
        pbest_obj[i] = f_lab(X[n_stages:(n_stages*2)][:,i], X[0:n_stages][:,i],V_A_in_array,V_C_in_array,start_array,goal_array,first_weight,sec_weight,third_weight,extra_sum,n_stages)
    #aqui ele pega nas coordenadas da partícula que deu valor menor na função objetivo
    gbest = pbest[:, pbest_obj.argmin()]
    #coloca o valor que a função objetivo deu para essas coordenadas 
    gbest_obj = pbest_obj.min()
    
    iterations=0
    # old_value=gbest_obj
    # new_value=0
    
    while(iterations<iteration_max):
        #defenir as constantes do algoritmo
        #old_value=new_value
        ''' subi c1 e c2 de 0.1 para 0.3 '''
        c1 = c2 = 0.2
        #incial era 0.8
        w = 0.6
        #fator aleatório
        r = np.random.rand(2)
        #atualização das partículas
        #substrainos as duas matrizes --- subtraimos a coluna de melhores valores a toda a coluna de X
        V = w * V + c1*r[0]*(pbest - X) + c2*r[1]*(gbest.reshape(-1,1)-X)
        X = X + V

        
        obj=np.zeros(n_particles)
        for i in range(0,n_particles):
            obj[i] = f_lab(X[n_stages:(n_stages*2)][:,i], X[0:n_stages][:,i],V_A_in_array,V_C_in_array,start_array,goal_array,first_weight,sec_weight,third_weight,extra_sum,n_stages)
            
        #vamos substituir as colunas de coordenadas cujos os valores na função deram inferiores
        pbest[:, (pbest_obj >= obj)] = X[:, (pbest_obj >= obj)]
        #atualizar o melhor valor atingido pela particula
        pbest_obj = np.array([pbest_obj, obj]).min(axis=0)
        gbest = pbest[:, pbest_obj.argmin()]
        gbest_obj = pbest_obj.min()
        
        #new_value=gbest_obj
        iterations+=1
        #print("-->",gbest_obj)
        
    return(gbest_obj,gbest)

In [190]:
def output_calc(gbest,input_pol,n_stages):
    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]
    delta_array_rad=delta_array*np.pi*2

    input_pol=np.reshape(input_pol,(3,1))
    
    M=wave_effect(alpha_array[n_stages-1],delta_array_rad[n_stages-1])
    i=n_stages-2
    while(i>=0): 
        M=np.dot(M,wave_effect(alpha_array[i],delta_array_rad[i]))
        i-=1
    
    output_pol=np.dot(M,input_pol)
    
    output_pol=np.reshape(output_pol,(1,3))
    output_pol=output_pol[0]
    
    return(output_pol)
    

In [191]:
def rise_time(delta_array,alpha_array,V_Ain_array,V_Cin_array,n_stages):
    V_A_measurment=np.zeros(n_stages)
    V_C_measurment=np.zeros(n_stages)

    V_pi_array=np.array([56.3,56,55.9,56.1,56.1,56,56.1,56])
    V_o_array=np.array([27.2,27,27,26.9,26.8,27.2,26.8,27.2])


    #V_A,bias e V_C,bias não têm efeito no processo de otimização 
    V_A_bias_array=np.array([-10.7,-9.3,-8.5,-10.9,-7.4,-7.6,-7.4,-7.6])
    V_C_bias_array=np.array([8.4,9.6,9.4,11.1,11.6,10.9,11.6,10.9])

    # V_Ain_array=V_A_in_array
    # V_Cin_array=V_C_in_array


    i=0
    while(i<n_stages):
        V_A_measurment[i]=V_A_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_A_bias_array[i])-V_Ain_array[i]
        i+=1
    i=0
    while(i<n_stages):
        V_C_measurment[i]=V_C_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_C_bias_array[i])-V_Cin_array[i]
        i+=1
    
    V_A_measurment_abs=abs(V_A_measurment)
    V_C_measurment_abs=abs(V_C_measurment)

    # print(np.max(V_A_measurment_abs))
    # print(np.max(V_C_measurment_abs))
    V_A_measurment_val=np.zeros(n_stages)
    V_C_measurment_val=np.zeros(n_stages)
    i=0
    while(i<n_stages):
        V_A_measurment_val[i]=V_A_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_A_bias_array[i])
        i+=1
    i=0
    while(i<n_stages):
        V_C_measurment_val[i]=V_C_value(alpha_array[i],delta_array[i],V_o_array[i],V_pi_array[i],V_C_bias_array[i])
        i+=1
    
    #print(V_Ain_array)
    
    return(V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs)

In [192]:
def particle_space(r_start_delta_array_A,r_start_delta_array_C,r_end_delta_array_A,r_end_delta_array_C,r_start_alpha_array_A,r_start_alpha_array_C,r_end_alpha_array_A,r_end_alpha_array_C,n_stages,n_particles):
    
    X_delta=np.array([])
    X_alpha=np.array([])
    i=0
    
    while(i<n_stages):
        
        start_delta_array_A=r_start_delta_array_A[i]
        start_delta_array_C=r_start_delta_array_C[i]
        end_delta_array_A=r_end_delta_array_A[i]
        end_delta_array_C=r_end_delta_array_C[i]
        start_alpha_array_A=r_start_alpha_array_A[i]
        start_alpha_array_C=r_start_alpha_array_C[i]
        end_alpha_array_A=r_end_alpha_array_A[i]
        end_alpha_array_C=r_end_alpha_array_C[i]
        
        X_delta_add,X_alpha_add=particle_distribution(n_particles/2,start_delta_array_A,end_delta_array_A,start_alpha_array_A,end_alpha_array_A)
        
        X_delta=np.concatenate((X_delta,X_delta_add))
        X_alpha=np.concatenate((X_alpha,X_alpha_add))
        
        X_delta_add,X_alpha_add=particle_distribution(n_particles/2,start_delta_array_C,end_delta_array_C,start_alpha_array_C,end_alpha_array_C)
        
        X_delta=np.concatenate((X_delta,X_delta_add))
        X_alpha=np.concatenate((X_alpha,X_alpha_add))
        
        i+=1
        
    ''' é possível que occoram erros no momento do reshape caso o o numero de partículas/len(...) não seja um número inteiro, parece-me que desde que
        o nº partículas seja /4 não deverá haver problemas
    '''
    X_delta=np.reshape(X_delta,(n_stages,n_particles))
    X_alpha=np.reshape(X_alpha,(n_stages,n_particles))
    
    return(X_delta,X_alpha)

In [193]:
import numpy as np

goal_array=np.array([1,0,0])
input_pol=np.array([0,0,1])

input_pol=np.reshape(input_pol,(3,1))

M1=wave_effect(np.pi,np.pi/2)
M2=wave_effect(np.pi,np.pi)
M3=wave_effect(np.pi/4,np.pi/2)

M=np.dot(M1,M3)
print(M1)
print(M3)
print(M)
calc_array=np.dot(M,input_pol)
calc_array=np.reshape(calc_array,(1,3))
calc_array=calc_array[0]


[[1.0, -1.224646799147353e-16, 1.2246467991473532e-16], [-1.224646799147353e-16, 6.123233995736767e-17, 1.0], [-1.2246467991473532e-16, -1.0, 6.123233995736766e-17]]
[[0.5000000000000001, 0.5, 0.7071067811865476], [0.5, 0.5000000000000001, -0.7071067811865476], [-0.7071067811865476, 0.7071067811865476, 6.123233995736766e-17]]
[[ 5.00000000e-01  5.00000000e-01  7.07106781e-01]
 [-7.07106781e-01  7.07106781e-01 -6.86610685e-17]
 [-5.00000000e-01 -5.00000000e-01  7.07106781e-01]]


## Starting H state

In [194]:
start = time.time()
V_pi_array=np.array([56.3,56,55.9,56.1,56.1,56,56.1,56])
V_o_array=np.array([27.2,27,27,26.9,26.8,27.2,26.8,27.2])
V_A_bias_array=np.array([-10.7,-9.3,-8.5,-10.9,-7.4,-7.6,-7.4,-7.6])
V_C_bias_array=np.array([8.4,9.6,9.4,11.1,11.6,10.9,11.6,10.9])

''' Parâmetros importantes '''
goal_array=np.array([1,0,0])
input_pol=np.array([ 0.55916189 , 0.12389344 , -0.91504942])
#input_pol=np.array([1,0,0])
reference_point=np.sqrt(input_pol[0]**2+input_pol[1]**2+input_pol[2]**2)

# np.random.seed(seed=0)
n_particles = 80
n_stages=6
''' -------------------------- '''

r_start_delta_array_A=[]
r_start_delta_array_C=[]
r_end_delta_array_A=[]
r_end_delta_array_C=[]
r_start_alpha_array_A=[]
r_start_alpha_array_C=[]
r_end_alpha_array_A=[]
r_end_alpha_array_C=[]

output_pol=np.zeros(3)
cond=1
while(cond==1):
    
    V_A_in_array=np.zeros(n_stages)
    V_C_in_array=np.zeros(n_stages)
    X_delta=np.array([])
    X_alpha=np.array([])
    i=0
    
    while(i<n_stages):
        
        start_delta_array_A,start_delta_array_C,end_delta_array_A,end_delta_array_C,start_alpha_array_A,start_alpha_array_C,end_alpha_array_A,end_alpha_array_C,V_A_in,V_C_in=interval_selection(V_pi_array[i],V_o_array[i],V_A_bias_array[i],V_C_bias_array[i])
        
        r_start_delta_array_A.append(start_delta_array_A)
        r_start_delta_array_C.append(start_delta_array_C)
        r_end_delta_array_A.append(end_delta_array_A)
        r_end_delta_array_C.append(end_delta_array_C)
        r_start_alpha_array_A.append(start_alpha_array_A)
        r_start_alpha_array_C.append(start_alpha_array_C)
        r_end_alpha_array_A.append(end_alpha_array_A)
        r_end_alpha_array_C.append(end_alpha_array_C)

        
        V_A_in_array[i]=V_A_in
        V_C_in_array[i]=V_C_in
        
        X_delta_add,X_alpha_add=particle_distribution(n_particles/2,start_delta_array_A,end_delta_array_A,start_alpha_array_A,end_alpha_array_A)
        
        X_delta=np.concatenate((X_delta,X_delta_add))
        X_alpha=np.concatenate((X_alpha,X_alpha_add))
        
        X_delta_add,X_alpha_add=particle_distribution(n_particles/2,start_delta_array_C,end_delta_array_C,start_alpha_array_C,end_alpha_array_C)
        
        X_delta=np.concatenate((X_delta,X_delta_add))
        X_alpha=np.concatenate((X_alpha,X_alpha_add))
        
        i+=1
        
    ''' é possível que occoram erros no momento do reshape caso o o numero de partículas/len(...) não seja um número inteiro, parece-me que desde que
        o nº partículas seja /4 não deverá haver problemas
    '''
    
    # X_delta_old=X_delta
    # X_alpha_old=X_alpha
    
    X_delta=np.reshape(X_delta,(n_stages,n_particles))
    X_alpha=np.reshape(X_alpha,(n_stages,n_particles))

    X = np.concatenate((X_delta, X_alpha))
    V = np.random.randn(n_stages*2, n_particles) * 0.1 #distribuição normal

    gbest_obj,gbest=PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,input_pol,250,2,10,10,0,n_stages)

    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]

    
    output_pol=output_calc(gbest,input_pol,n_stages)

    V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs=rise_time(delta_array,alpha_array,V_A_in_array,V_C_in_array,n_stages)
    
    V_A_max=np.max(V_A_measurment_abs)
    V_C_max=np.max(V_C_measurment_abs)
    
    if(abs(output_pol[0]-reference_point)<1e-4 and V_A_max<3 and V_C_max<3):
        cond=0
    
    print("V_A diff -> ",V_A_max)
    print("V_C diff -> ",V_C_max)
    print("Output pol", output_pol)
    print(abs(output_pol[0]-reference_point))
    print("gbest ->",gbest_obj)

V_A diff ->  2.217009424468735
V_C diff ->  1.933878092861498
Output pol [ 1.07950315e+00 -6.82209385e-07  1.66867940e-09]
2.156053113822054e-13
gbest -> 0.44101865110590055


In [195]:

print("Os valores de delta para o estado H são:")
print(delta_array)
print("Os valores de alpha para o estado H são:")
print(alpha_array)
print("Os valores de V_A para os estado H são:")
print(V_A_measurment_val)
print("Os valores de V_C para os estado H são:")
print(V_C_measurment_val)
print("----------")
print("Estado de polarização na saída")
print("S1=%f, S2=%f, S3=%f"%(output_pol[0],output_pol[1],output_pol[2]))

V_A_measurment_val_H=V_A_measurment_val
V_C_measurment_val_H=V_C_measurment_val

file1 = open("registo_alpha.txt", "w")
file2 = open("registo_delta.txt", "w")
file1.write("H\n")
file2.write("H\n")
str_alpha=repr(alpha_array)
file1.write(str_alpha+"\n")
str_delta=repr(delta_array)
file2.write(str_delta+"\n")
file1.close()
file2.close()

Os valores de delta para o estado H são:
[0.03553893 0.03212831 0.01849859 0.02294987 0.02608799 0.03541481]
Os valores de alpha para o estado H são:
[4.57289489 4.44343606 4.4497941  4.36837577 4.4705977  4.34265005]
Os valores de V_A para os estado H são:
[-12.33633719 -10.49447405  -9.19624858 -11.6281318   -8.40720797
  -8.67968817]
Os valores de V_C para os estado H são:
[6.20726015 7.44935918 8.16688782 9.50341191 9.89192727 8.38694451]
----------
Estado de polarização na saída
S1=1.079503, S2=-0.000001, S3=0.000000


## H $\rightarrow$ V

In [196]:
output_pol=np.zeros(3)
V_A_max=100
V_C_max=100
cond=1
while(cond==1):
    
    ''' Important parameters '''
    goal_array=np.array([-1,0,0])
    ''' -------------------------- '''
    X_delta,X_alpha=particle_space(r_start_delta_array_A,r_start_delta_array_C,r_end_delta_array_A,r_end_delta_array_C,r_start_alpha_array_A,r_start_alpha_array_C,r_end_alpha_array_A,r_end_alpha_array_C,n_stages,n_particles)
    
    X = np.concatenate((X_delta, X_alpha))

    V = np.random.randn(n_stages*2, n_particles) * 0.1 #distribuição normal

    V_A_in_array=V_A_measurment_val_H
    V_C_in_array=V_C_measurment_val_H


    #2 para 3 depois da run
    gbest_obj,gbest=PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,input_pol,150,2,10,10,0,n_stages)
    ''' 2 stages '''
    # gbest_obj,gbest=PSO(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,0,1,150,20,10,10,n_stages)

    # gbest_obj,gbest=PSO(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,0,1,150,15,5,5)

    output_pol=output_calc(gbest,input_pol,n_stages)
        
    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]
    
    V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs=rise_time(delta_array,alpha_array,V_A_in_array,V_C_in_array,n_stages)
        
    V_A_max=np.max(V_A_measurment_abs)
    V_C_max=np.max(V_C_measurment_abs)
    ''' 8 stages '''
    # if(abs(output_pol[0]+1)<1e-4 and V_A_max<7 and V_C_max<7):
    #     cond=0
    ''' 6 stages '''
    if(abs(output_pol[0]+reference_point)<1e-4 and V_A_max<7 and V_C_max<7):
        cond=0
    ''' 3 stages '''
    # if(abs(output_pol[0]+1)<1e-4 and V_A_max<13.5 and V_C_max<13.5):
    ''' 2 stages '''
    # if(abs(output_pol[0]+1)<1e-4 and V_A_max<20 and V_C_max<20):
        #cond=0  
    # if(abs(output_pol[0]+1)<1e-4):
        
    #     cond=0

    print("V_A diff -> ",V_A_max)
    print("V_C diff -> ",V_C_max)
    print("Output pol", output_pol)
    print("gbest ->",gbest_obj)

V_A diff ->  6.160636417129522
V_C diff ->  6.481845037405516
Output pol [-1.00036058e+00  5.22497869e-12  4.05716342e-01]
gbest -> 3.177580603735146
V_A diff ->  4.72073535704799
V_C diff ->  6.163256531961961
Output pol [-1.07277233e+00 -7.85652061e-06 -1.20360201e-01]
gbest -> 2.7293222106948947
V_A diff ->  7.409183609106291
V_C diff ->  5.8102250980231025
Output pol [-1.00021535e+00 -2.36975019e-07  4.06074243e-01]
gbest -> 2.944091361555637
V_A diff ->  7.279288086946002
V_C diff ->  5.790416816487075
Output pol [-1.07950315e+00 -5.85577907e-14  2.83412693e-12]
gbest -> 2.7801437115932925
V_A diff ->  6.239343747799974
V_C diff ->  6.549128638189529
Output pol [-1.07950304e+00  8.08334274e-05 -4.68371492e-04]
gbest -> 2.8835766502406734


In [197]:
print("Os valores de delta para o estado V são:")
print(delta_array)
print("Os valores de alpha para o estado V são:")
print(alpha_array)
print("Os valores de V_A para os estado V são:")
print(V_A_measurment_val)
print("Os valores de V_C para os estado V são:")
print(V_C_measurment_val)
print("----------")
print("Estado de polarização na saída")
print("S1=%f, S2=%f, S3=%f"%(output_pol[0],output_pol[1],output_pol[2]))   


file1 = open("registo_alpha.txt", "a")
file2 = open("registo_delta.txt", "a")
file1.write("V\n")
file2.write("V\n")
str_alpha=repr(alpha_array)
file1.write(str_alpha+"\n")
str_delta=repr(delta_array)
file2.write(str_delta+"\n")
file1.close()
file2.close()

print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_measurment_abs)>np.max(V_C_measurment_abs)):
    print("Rise time será de",((np.max(V_A_measurment_abs)*1000)/5876),"ns")

else:
    print("Rise time será de",((np.max(V_C_measurment_abs)*1000)/5876),"ns")

V_A_measurment_val_V=V_A_measurment_val
V_C_measurment_val_V=V_C_measurment_val

Os valores de delta para o estado V são:
[0.07464343 0.04867513 0.01680305 0.08478769 0.08259636 0.03233668]
Os valores de alpha para o estado V são:
[1.70960468 1.57769384 1.7628238  1.68263472 1.47715042 1.80668952]
Os valores de V_A para os estado V são:
[-6.09699344 -6.65280428 -7.43005048 -5.83605899 -3.42552201 -5.46638408]
Os valores de V_C para os estado V são:
[11.84008597 12.20959344 10.11142339 15.10221883 16.44105591 12.18718095]
----------
Estado de polarização na saída
S1=-1.079503, S2=0.000081, S3=-0.000468
RESULTADOS PARA PASSAGEM:
Rise time será de 1.1145555885278302 ns


## H $\rightarrow$ +45

In [198]:
output_pol=np.zeros(3)
V_A_max=100
V_C_max=100
cond=1

V_A_in_array=np.concatenate((V_A_measurment_val_H, V_A_measurment_val_V))
V_C_in_array=np.concatenate((V_C_measurment_val_H, V_C_measurment_val_V))

V_A_in_array=np.reshape(V_A_in_array,(2,n_stages))
V_C_in_array=np.reshape(V_C_in_array,(2,n_stages))
# print(len(V_A_in_array))
# print(len(V_C_in_array))
    
while(cond==1):
    
    ''' Parâmetros importantes '''
    goal_array=np.array([0,1,0])
    ''' -------------------------- '''
    
    X_delta,X_alpha=particle_space(r_start_delta_array_A,r_start_delta_array_C,r_end_delta_array_A,r_end_delta_array_C,r_start_alpha_array_A,r_start_alpha_array_C,r_end_alpha_array_A,r_end_alpha_array_C,n_stages,n_particles)

    X = np.concatenate((X_delta, X_alpha))

    V = np.random.randn(n_stages*2, n_particles) * 0.1 #distribuição normal
    

    ''' quero adicionar uma renovação de partículas '''
    
    gbest_obj,gbest=PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,input_pol,150,3,10,10,1,n_stages)
    ''' 2 stages '''
    # gbest_obj,gbest=PSO(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,0,1,150,25,10,10,n_stages)


    output_pol=output_calc(gbest,input_pol,n_stages)
    
    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]
    
    V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs=rise_time(delta_array,alpha_array,V_A_in_array[0],V_C_in_array[0],n_stages)
        
    V_A_max=np.max(V_A_measurment_abs)
    V_C_max=np.max(V_C_measurment_abs)
    ''' 8 stages '''
    # if(abs(output_pol[1]-1)<1e-4 and V_A_max<7 and V_C_max<7):
    #     cond=0
    ''' 6 stages '''
    if(abs(output_pol[1]-reference_point)<1e-4 and V_A_max<8.5 and V_C_max<8.5):
        cond=0
    ''' 3 stages '''
    #if(abs(output_pol[1]-1)<1e-4 and V_A_max<13.5 and V_C_max<13.5):
    ''' 2 stages '''
    # if(abs(output_pol[1]-1)<1e-4 and V_A_max<20 and V_C_max<20):

    #     cond=0
    # if(abs(output_pol[1]-1)<1e-4):
    #     cond=0
    
    print("V_A diff -> ",V_A_max)
    print("V_C diff -> ",V_C_max)
    print("Output pol", output_pol)
    print("gbest ->",gbest_obj)

V_A diff ->  5.171374195964707
V_C diff ->  9.748881667883609
Output pol [-5.41878263e-13  1.07950315e+00  1.99345001e-11]
gbest -> 3.5635645245469343
V_A diff ->  3.9289279414840284
V_C diff ->  8.787767923273526
Output pol [-6.31859657e-08  1.07950315e+00  4.33961218e-06]
gbest -> 3.314340303578529
V_A diff ->  3.974366818998295
V_C diff ->  16.33893574408344
Output pol [5.63678736e-04 1.07950300e+00 3.80879512e-06]
gbest -> 2.5792539550519242
V_A diff ->  3.288080908780767
V_C diff ->  12.242239767630288
Output pol [-0.01762464  1.07529824  0.09354206]
gbest -> 1.9910118340447405
V_A diff ->  1.7895655763184717
V_C diff ->  12.288442250302849
Output pol [-2.19675675e-08  1.07882642e+00  3.82178345e-02]
gbest -> 1.7158110597101837
V_A diff ->  1.3439017315840918
V_C diff ->  7.801117025810495
Output pol [-1.69975811e-07  1.07950315e+00 -4.47598228e-07]
gbest -> 1.153163006212083


In [199]:
print("Os valores de delta para o estado +45 são:")
print(delta_array)
print("Os valores de alpha para o estado +45 são:")
print(alpha_array)
print("Os valores de V_A para os estado +45 são:")
print(V_A_measurment_val)
print("Os valores de V_B para os estado +45 são:")
print(V_C_measurment_val)
print("----------")
print("Estado de polarização na saída")
print("S1=%f, S2=%f, S3=%f"%(output_pol[0],output_pol[1],output_pol[2]))  

file1 = open("registo_alpha.txt", "a")
file2 = open("registo_delta.txt", "a")
file1.write("+45\n")
file2.write("+45\n")
str_alpha=repr(alpha_array)
file1.write(str_alpha+"\n")
str_delta=repr(delta_array)
file2.write(str_delta+"\n")
file1.close()
file2.close() 


Os valores de delta para o estado +45 são:
[0.07271577 0.02441987 0.08767016 0.02696741 0.04726887 0.02653106]
Os valores de alpha para o estado +45 são:
[0.46836974 0.75443875 0.49861322 0.60182176 0.52683456 0.73136461]
Os valores de V_A para os estado +45 são:
[-12.56725969  -9.3933139  -10.54015031 -11.32567887  -8.41831067
  -7.74182905]
Os valores de V_B para os estado +45 são:
[13.83875434 11.49957888 15.96800485 13.16845826 15.16610845 12.96973451]
----------
Estado de polarização na saída
S1=-0.000000, S2=1.079503, S3=-0.000000


In [200]:
print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_measurment_abs)>np.max(V_C_measurment_abs)):
    print("Rise time será de",((np.max(V_A_measurment_abs)*1000)/5876),"ns")

else:
    print("Rise time será de",((np.max(V_C_measurment_abs)*1000)/5876),"ns")
    
V_A_measurment_val_p45=V_A_measurment_val
V_C_measurment_val_p45=V_C_measurment_val

RESULTADOS PARA PASSAGEM:
Rise time será de 1.3276237280140393 ns


## H $\rightarrow$ -45

In [201]:
output_pol=np.zeros(3)
V_A_max=100
V_C_max=100
cond=1

V_A_in_array=np.concatenate((V_A_measurment_val_H, V_A_measurment_val_V))
V_C_in_array=np.concatenate((V_C_measurment_val_H, V_C_measurment_val_V))

V_A_in_array=np.concatenate((V_A_in_array, V_A_measurment_val_p45))
V_C_in_array=np.concatenate((V_C_in_array, V_C_measurment_val_p45))


V_A_in_array=np.reshape(V_A_in_array,(3,n_stages))
V_C_in_array=np.reshape(V_C_in_array,(3,n_stages))

while(cond==1):
    it_start=time.time()
    ''' Parâmetros importantes '''
    goal_array=np.array([0,-1,0])
    ''' -------------------------- '''

    X_delta,X_alpha=particle_space(r_start_delta_array_A,r_start_delta_array_C,r_end_delta_array_A,r_end_delta_array_C,r_start_alpha_array_A,r_start_alpha_array_C,r_end_alpha_array_A,r_end_alpha_array_C,n_stages,n_particles)


    X = np.concatenate((X_delta, X_alpha))

    V = np.random.randn(n_stages*2, n_particles) * 0.1 #distribuição normal
    

    ''' quero adicionar uma renovação de partículas '''
    
    gbest_obj,gbest=PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,input_pol,150,2,10,10,1,n_stages)
    ''' 2 stages '''
    # gbest_obj,gbest=PSO(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,0,1,150,25,10,10,n_stages)


    output_pol=output_calc(gbest,input_pol,n_stages)
    
    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]
    
    V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs=rise_time(delta_array,alpha_array,V_A_in_array[0],V_C_in_array[0],n_stages)
        
    V_A_max=np.max(V_A_measurment_abs)
    V_C_max=np.max(V_C_measurment_abs)
    ''' 8 stages '''
    # if(abs(output_pol[1]+1)<1e-4 and V_A_max<7 and V_C_max<7):
    #     cond=0
    ''' 6 stages '''
    if(abs(output_pol[1]+reference_point)<1e-4 and V_A_max<6 and V_C_max<6):
        cond=0
    ''' 3 stages '''
    #if(abs(output_pol[1]+1)<1e-4 and V_A_max<13.5 and V_C_max<13.5):
    ''' 2 stages '''
    # if(abs(output_pol[1]+1)<1e-4 and V_A_max<20 and V_C_max<20):

    #     cond=0  
    # if(abs(output_pol[1]+1)<1e-4):
        
    #     cond=0
    
    it_stop=time.time()
    print("V_A diff -> ",V_A_max)
    print("V_C diff -> ",V_C_max)
    print("Output pol", output_pol)
    print("gbest ->",gbest_obj)
    print("time diff ->",it_stop-it_start)

V_A diff ->  7.082131957733514
V_C diff ->  1.7947175615231759
Output pol [ 3.55318932e-07 -1.07950315e+00 -8.29929458e-09]
gbest -> 0.8723972036418796
time diff -> 12.999156951904297
V_A diff ->  6.813557896703144
V_C diff ->  1.190358668650429
Output pol [-1.28279169e-08 -1.07950315e+00  1.34857648e-07]
gbest -> 0.8411295967813073
time diff -> 16.47058868408203
V_A diff ->  7.430371483853373
V_C diff ->  1.528337155533733
Output pol [-3.79640920e-10 -1.07950315e+00  1.23367170e-09]
gbest -> 0.8590881624686254
time diff -> 13.022764682769775
V_A diff ->  7.41598037529195
V_C diff ->  1.2809512021957712
Output pol [ 1.35344226e-10 -1.07950315e+00 -3.61875596e-09]
gbest -> 0.8339642975187045
time diff -> 12.086569786071777
V_A diff ->  8.103470117069604
V_C diff ->  2.0584157533147938
Output pol [-1.40609952e-14 -1.07950315e+00  5.23985079e-09]
gbest -> 0.9546154599952765
time diff -> 12.783438444137573
V_A diff ->  8.417487856903763
V_C diff ->  1.1856757809714802
Output pol [ 5.182806

In [202]:

print("Os valores de delta para o estado -45 são:")
print(delta_array)
print("Os valores de alpha para o estado -45 são:")
print(alpha_array)
print("Os valores de V_A para os estado -45 são:")
print(V_A_measurment_val)
print("Os valores de V_C para os estado -45 são:")
print(V_C_measurment_val)
print("----------")
print("Estado de polarização na saída")
print("S1=%f, S2=%f, S3=%f"%(output_pol[0],output_pol[1],output_pol[2]))   

file1 = open("registo_alpha.txt", "a")
file2 = open("registo_delta.txt", "a")
file1.write("-45\n")
file2.write("-45\n")
str_alpha=repr(alpha_array)
file1.write(str_alpha+"\n")
str_delta=repr(delta_array)
file2.write(str_delta+"\n")
file1.close()
file2.close()

Os valores de delta para o estado -45 são:
[0.04565401 0.0619087  0.04457535 0.06202406 0.03888721 0.05312143]
Os valores de alpha para o estado -45 são:
[2.71854771 2.66522345 2.69171689 2.71201763 2.83929723 2.71448541]
Os valores de V_A para os estado -45 são:
[-7.33666565 -4.68611249 -5.20944241 -6.34682726 -4.69681167 -3.69536026]
Os valores de V_C para os estado -45 são:
[ 7.07587617  8.0520797   8.20288804  9.3263569  10.13788765  9.38950463]
----------
Estado de polarização na saída
S1=-0.000000, S2=-1.079503, S3=0.000000


In [203]:
print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_measurment_abs)>np.max(V_C_measurment_abs)):
    print("Rise time será de",((np.max(V_A_measurment_abs)*1000)/5876),"ns")

else:
    print("Rise time será de",((np.max(V_C_measurment_abs)*1000)/5876),"ns")

V_A_measurment_val_n45=V_A_measurment_val
V_C_measurment_val_n45=V_C_measurment_val

RESULTADOS PARA PASSAGEM:
Rise time será de 0.9884890326797537 ns


## H $\rightarrow$ $\curvearrowright$

In [204]:
output_pol=np.zeros(3)
V_A_max=100
V_C_max=100
cond=1

V_A_in_array=np.concatenate((V_A_measurment_val_H, V_A_measurment_val_V))
V_C_in_array=np.concatenate((V_C_measurment_val_H, V_C_measurment_val_V))

V_A_in_array=np.concatenate((V_A_in_array, V_A_measurment_val_p45))
V_C_in_array=np.concatenate((V_C_in_array, V_C_measurment_val_p45))

V_A_in_array=np.concatenate((V_A_in_array, V_A_measurment_val_n45))
V_C_in_array=np.concatenate((V_C_in_array, V_C_measurment_val_n45))


V_A_in_array=np.reshape(V_A_in_array,(4,n_stages))
V_C_in_array=np.reshape(V_C_in_array,(4,n_stages))


while(cond==1):
    
    ''' Parâmetros importantes '''
    goal_array=np.array([0,0,1])
    ''' -------------------------- '''

    X_delta,X_alpha=particle_space(r_start_delta_array_A,r_start_delta_array_C,r_end_delta_array_A,r_end_delta_array_C,r_start_alpha_array_A,r_start_alpha_array_C,r_end_alpha_array_A,r_end_alpha_array_C,n_stages,n_particles)


    X = np.concatenate((X_delta, X_alpha))

    V = np.random.randn(n_stages*2, n_particles) * 0.1 #distribuição normal
    

    ''' quero adicionar uma renovação de partículas '''
    
    gbest_obj,gbest=PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,input_pol,150,2,10,10,1,n_stages)
    ''' 2 stages '''
    # gbest_obj,gbest=PSO(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,0,1,150,25,10,10,n_stages)


    output_pol=output_calc(gbest,input_pol,n_stages)
    
    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]
    
    V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs=rise_time(delta_array,alpha_array,V_A_in_array[0],V_C_in_array[0],n_stages)
    
    
        
    V_A_max=np.max(V_A_measurment_abs)
    V_C_max=np.max(V_C_measurment_abs)

    V_A_measurment_val_ex,V_C_measurment_val_ex,V_A_measurment_abs_ex,V_C_measurment_abs_ex=rise_time(delta_array,alpha_array,V_A_measurment_val_p45,V_C_measurment_val_p45,n_stages)
    
    V_A_max_ex=np.max(V_A_measurment_abs_ex)
    V_C_max_ex=np.max(V_C_measurment_abs_ex)
    # if(abs(output_pol[2]-1)<1e-4 and V_A_max<7 and V_C_max<7):
    #     cond=0
    if(abs(output_pol[2]-reference_point)<1e-4 and V_A_max<8.06 and V_C_max<8.06):
        cond=0
    #if(abs(output_pol[2]-1)<1e-4 and V_A_max<13.5 and V_C_max<13.5):
    # if(abs(output_pol[2]-1)<1e-4 and V_A_max<20 and V_C_max<20):
    
   
    #     cond=0  
    # if(abs(output_pol[2]-1)<1e-4):
    #     cond=0
    #print(V_A_max_ex)
    #print(V_C_max_ex)
    print("V_A diff -> ",V_A_max)
    print("V_C diff -> ",V_C_max)
    print("Output pol", output_pol)
    print("gbest ->",gbest_obj)

V_A diff ->  7.498639876370202
V_C diff ->  1.1621096588690394
Output pol [1.23897071e-09 4.07454991e-09 1.07950315e+00]
gbest -> 1.1379798141014434


In [205]:

print("Os valores de delta para o estado +circ são:")
print(delta_array)
print("Os valores de alpha para o estado +circ são:")
print(alpha_array)
print("Os valores de V_A para os estado +circ são:")
print(V_A_measurment_val)
print("Os valores de V_C para os estado +circ são:")
print(V_C_measurment_val)
print("----------")
print("Estado de polarização na saída")
print("S1=%f, S2=%f, S3=%f"%(output_pol[0],output_pol[1],output_pol[2]))   

file1 = open("registo_alpha.txt", "a")
file2 = open("registo_delta.txt", "a")
file1.write("circ+\n")
file2.write("circ+\n")
str_alpha=repr(alpha_array)
file1.write(str_alpha+"\n")
str_delta=repr(delta_array)
file2.write(str_delta+"\n")
file1.close()
file2.close()

Os valores de delta para o estado +circ são:
[0.08841451 0.12110116 0.03484252 0.082084   0.0473504  0.04098676]
Os valores de alpha para o estado +circ são:
[5.30538941 5.12029123 5.18416305 5.08391188 5.11470545 5.03162477]
Os valores de V_A para os estado +circ são:
[-17.47038594 -17.99311392 -11.06113266 -16.68657566 -10.77543942
 -10.43737268]
Os valores de V_C para os estado +circ são:
[ 7.19324738  6.28724952  8.60919651  8.65691176 10.30475909  9.50332003]
----------
Estado de polarização na saída
S1=0.000000, S2=0.000000, S3=1.079503


In [206]:
print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_measurment_abs)>np.max(V_C_measurment_abs)):
    print("Rise time será de",((np.max(V_A_measurment_abs)*1000)/5876),"ns")

else:
    print("Rise time será de",((np.max(V_C_measurment_abs)*1000)/5876),"ns")

V_A_measurment_val_pcirc=V_A_measurment_val
V_C_measurment_val_pcirc=V_C_measurment_val

RESULTADOS PARA PASSAGEM:
Rise time será de 1.2761470177621175 ns


## H $\rightarrow$ $\curvearrowleft$


In [207]:
output_pol=np.zeros(3)
V_A_max=100
V_C_max=100
cond=1

V_A_in_array=np.concatenate((V_A_measurment_val_H, V_A_measurment_val_V))
V_C_in_array=np.concatenate((V_C_measurment_val_H, V_C_measurment_val_V))

V_A_in_array=np.concatenate((V_A_in_array, V_A_measurment_val_p45))
V_C_in_array=np.concatenate((V_C_in_array, V_C_measurment_val_p45))

V_A_in_array=np.concatenate((V_A_in_array, V_A_measurment_val_n45))
V_C_in_array=np.concatenate((V_C_in_array, V_C_measurment_val_n45))

V_A_in_array=np.concatenate((V_A_in_array, V_A_measurment_val_pcirc))
V_C_in_array=np.concatenate((V_C_in_array, V_C_measurment_val_pcirc))


V_A_in_array=np.reshape(V_A_in_array,(5,n_stages))
V_C_in_array=np.reshape(V_C_in_array,(5,n_stages))

while(cond==1):
    
    ''' Parâmetros importantes '''
    goal_array=np.array([0,0,-1])
    ''' -------------------------- '''
    
    X_delta,X_alpha=particle_space(r_start_delta_array_A,r_start_delta_array_C,r_end_delta_array_A,r_end_delta_array_C,r_start_alpha_array_A,r_start_alpha_array_C,r_end_alpha_array_A,r_end_alpha_array_C,n_stages,n_particles)


    X = np.concatenate((X_delta, X_alpha))

    V = np.random.randn(n_stages*2, n_particles) * 0.1 #distribuição normal
    

    ''' quero adicionar uma renovação de partículas '''
    
    gbest_obj,gbest=PSO_lab(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,input_pol,150,2,10,10,1,n_stages)
    ''' 2 stages '''
    #gbest_obj,gbest=PSO(X,V,n_particles,goal_array,V_A_in_array,V_C_in_array,0,1,150,25,10,10,n_stages)


    output_pol=output_calc(gbest,input_pol,n_stages)
    
    delta_array=gbest[0:n_stages]
    alpha_array=gbest[n_stages:(n_stages*2)]
    
    V_A_measurment_val,V_C_measurment_val,V_A_measurment_abs,V_C_measurment_abs=rise_time(delta_array,alpha_array,V_A_in_array[0],V_C_in_array[0],n_stages)
    # V_A_measurment_val_ex,V_C_measurment_val_ex,V_A_measurment_abs_ex,V_C_measurment_abs_ex=rise_time(delta_array,alpha_array,V_A_measurment_val_n45,V_C_measurment_val_n45,n_stages)

    V_A_max=np.max(V_A_measurment_abs)
    V_C_max=np.max(V_C_measurment_abs)
    
    # V_A_max_ex=np.max(V_A_measurment_abs_ex)
    # V_C_max_ex=np.max(V_C_measurment_abs_ex)
    
    # n45 para circular negativa
    ''' 8 stages '''
    # if(abs(output_pol[2]+1)<1e-4 and V_A_max<7.8 and V_C_max<7.8 and V_A_max_ex<7.8 and V_C_max_ex<7.8):
    #     cond=0
    # if(abs(output_pol[2]+1)<1e-4 and V_A_max<7 and V_C_max<7):
    #     cond=0
    ''' 6 stages '''
    # if(abs(output_pol[2]+reference_point)<1e-4 and V_A_max<8.8 and V_C_max<8.8):
    #     cond=0
    if(abs(output_pol[2]+reference_point)<1e-4 and V_A_max<5 and V_C_max<5 ):
        cond=0
    ''' 3 stages '''
    #if(abs(output_pol[2]+1)<1e-4 and V_A_max<13.5 and V_C_max<13.5 and V_A_max_ex<13.5 and V_C_max_ex<13.5):
    ''' 2 stages '''
    # if(abs(output_pol[2]+1)<1e-4 and V_A_max<20 and V_C_max<20 and V_A_max_ex<20 and V_C_max_ex<20):

    #     cond=0
    # if(abs(output_pol[2]+1)<1e-4):
        
    #     cond=0
    # print(V_A_max_ex)
    # print(V_C_max_ex)    
    print("V_A diff -> ",V_A_max)
    print("V_C diff -> ",V_C_max)
    print("Output pol", output_pol)
    print("gbest ->",gbest_obj)

V_A diff ->  5.457539337790931
V_C diff ->  4.860615021301132
Output pol [ 8.23867925e-09 -1.16664824e-06 -1.07950315e+00]
gbest -> 0.4631741553752556
V_A diff ->  4.836078664918034
V_C diff ->  4.010929073354045
Output pol [-1.04984313e-12  6.82458815e-12 -1.07950315e+00]
gbest -> 0.450741619474477


In [208]:

print("Os valores de delta para o estado -circ são:")
print(delta_array)
print("Os valores de alpha para o estado -circ são:")
print(alpha_array)
print("Os valores de V_A para os estado -circ são:")
print(V_A_measurment_val)
print("Os valores de V_C para os estado -circ são:")
print(V_C_measurment_val)
print("----------")
print("Estado de polarização na saída")
print("S1=%f, S2=%f, S3=%f"%(output_pol[0],output_pol[1],output_pol[2]))   

file1 = open("registo_alpha.txt", "a")
file2 = open("registo_delta.txt", "a")
file1.write("circ-\n")
file2.write("circ-\n")
str_alpha=repr(alpha_array)
file1.write(str_alpha+"\n")
str_delta=repr(delta_array)
file2.write(str_delta+"\n")
file1.close()
file2.close()

Os valores de delta para o estado -circ são:
[ 0.00702162  0.03848801  0.00081359  0.00945274  0.05780356 -0.02428879]
Os valores de alpha para o estado -circ são:
[1.80681051 1.66760677 1.74954598 1.77987312 1.83125736 1.71214007]
Os valores de V_A para os estado -circ são:
[-10.23617672  -7.02304666  -8.44867968 -10.2924506   -3.57112931
  -9.09974571]
Os valores de V_C para os estado -circ são:
[ 8.67895    11.46028825  9.43514774 11.48741521 13.7586701   9.78347892]
----------
Estado de polarização na saída
S1=-0.000000, S2=0.000000, S3=-1.079503


In [209]:
print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_measurment_abs)>np.max(V_C_measurment_abs)):
    print("Rise time será de",((np.max(V_A_measurment_abs)*1000)/5876),"ns")

else:
    print("Rise time será de",((np.max(V_C_measurment_abs)*1000)/5876),"ns")

V_A_measurment_val_ncirc=V_A_measurment_val
V_C_measurment_val_ncirc=V_C_measurment_val

RESULTADOS PARA PASSAGEM:
Rise time será de 0.823022237052082 ns


## V $\rightarrow$ +45

In [210]:
V_A_Calc=V_A_measurment_val_V-V_A_measurment_val_p45
V_C_Calc=V_C_measurment_val_V-V_C_measurment_val_p45

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print(V_A_Calc)
print(V_C_Calc)

print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Rise time será de",((np.max(V_A_Calc)*1000)/5876),"ns",np.max(V_A_Calc))

else:
    print("Rise time será de",((np.max(V_C_Calc)*1000)/5876),"ns",np.max(V_C_Calc))

[6.47026625 2.74050962 3.11009983 5.48961987 4.99278866 2.27544497]
[1.99866836 0.71001456 5.85658146 1.93376057 1.27494746 0.78255356]
RESULTADOS PARA PASSAGEM:
Rise time será de 1.1011344876038702 ns 6.470266249160342


## V $\rightarrow$ -45

In [211]:
V_A_Calc=V_A_measurment_val_V-V_A_measurment_val_n45
V_C_Calc=V_C_measurment_val_V-V_C_measurment_val_n45

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Rise time será de",((np.max(V_A_Calc)*1000)/5876),"ns",np.max(V_A_Calc))

else:
    print("Rise time será de",((np.max(V_C_Calc)*1000)/5876),"ns",np.max(V_C_Calc))

RESULTADOS PARA PASSAGEM:
Rise time será de 1.0726971179773628 ns 6.303168265234984


## V $\rightarrow$ circ+

In [212]:
V_A_Calc=V_A_measurment_val_V-V_A_measurment_val_pcirc
V_C_Calc=V_C_measurment_val_V-V_C_measurment_val_pcirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Rise time será de",((np.max(V_A_Calc)*1000)/5876),"ns",np.max(V_A_Calc))

else:
    print("Rise time será de",((np.max(V_C_Calc)*1000)/5876),"ns",np.max(V_C_Calc))

RESULTADOS PARA PASSAGEM:
Rise time será de 1.9355671365179914 ns 11.373392494179718


## V $\rightarrow$ circ-

In [213]:
V_A_Calc=V_A_measurment_val_V-V_A_measurment_val_ncirc
V_C_Calc=V_C_measurment_val_V-V_C_measurment_val_ncirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Rise time será de",((np.max(V_A_Calc)*1000)/5876),"ns",np.max(V_A_Calc))

else:
    print("Rise time será de",((np.max(V_C_Calc)*1000)/5876),"ns",np.max(V_C_Calc))

RESULTADOS PARA PASSAGEM:
Rise time será de 0.7584056506493605 ns 4.456391603215642


## +45  $\rightarrow$ -45

In [214]:
V_A_Calc=V_A_measurment_val_p45-V_A_measurment_val_n45
V_C_Calc=V_C_measurment_val_p45-V_C_measurment_val_n45

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Rise time será de",((np.max(V_A_Calc)*1000)/5876),"ns",np.max(V_A_Calc))

else:
    print("Rise time será de",((np.max(V_C_Calc)*1000)/5876),"ns",np.max(V_C_Calc))

RESULTADOS PARA PASSAGEM:
Rise time será de 1.3214970747816122 ns 7.765116811416753


## +45 $\rightarrow$ circ+

In [215]:
V_A_Calc=V_A_measurment_val_p45-V_A_measurment_val_pcirc
V_C_Calc=V_C_measurment_val_p45-V_C_measurment_val_pcirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Rise time será de",((np.max(V_A_Calc)*1000)/5876),"ns",np.max(V_A_Calc))

else:
    print("Rise time será de",((np.max(V_C_Calc)*1000)/5876),"ns",np.max(V_C_Calc))

RESULTADOS PARA PASSAGEM:
Rise time será de 1.463546634672624 ns 8.59980002533634


## +45 $\rightarrow$ circ-

In [216]:
V_A_Calc=V_A_measurment_val_p45-V_A_measurment_val_ncirc
V_C_Calc=V_C_measurment_val_p45-V_C_measurment_val_ncirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Rise time será de",((np.max(V_A_Calc)*1000)/5876),"ns",np.max(V_A_Calc))

else:
    print("Rise time será de",((np.max(V_C_Calc)*1000)/5876),"ns",np.max(V_C_Calc))

RESULTADOS PARA PASSAGEM:
Rise time será de 1.1117864374716935 ns 6.532857106583672


## -45 $\rightarrow$ circ+

In [217]:
V_A_Calc=V_A_measurment_val_n45-V_A_measurment_val_pcirc
V_C_Calc=V_C_measurment_val_n45-V_C_measurment_val_pcirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Rise time será de",((np.max(V_A_Calc)*1000)/5876),"ns",np.max(V_A_Calc))

else:
    print("Rise time será de",((np.max(V_C_Calc)*1000)/5876),"ns",np.max(V_C_Calc))

RESULTADOS PARA PASSAGEM:
Rise time será de 2.2646360504418714 ns 13.307001432396437


## -45 $\rightarrow$ circ-

In [218]:
V_A_Calc=V_A_measurment_val_n45-V_A_measurment_val_ncirc
V_C_Calc=V_C_measurment_val_n45-V_C_measurment_val_ncirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Rise time será de",((np.max(V_A_Calc)*1000)/5876),"ns",np.max(V_A_Calc))

else:
    print("Rise time será de",((np.max(V_C_Calc)*1000)/5876),"ns",np.max(V_C_Calc))

RESULTADOS PARA PASSAGEM:
Rise time será de 0.9197388450952156 ns 5.404385453779486


## circ+ $\rightarrow$ circ-

In [219]:
V_A_Calc=V_A_measurment_val_pcirc-V_A_measurment_val_ncirc
V_C_Calc=V_C_measurment_val_pcirc-V_C_measurment_val_ncirc

V_A_Calc=abs(V_A_Calc)
V_C_Calc=abs(V_C_Calc)

print("RESULTADOS PARA PASSAGEM:")
if(np.max(V_A_Calc)>np.max(V_C_Calc)):
    print("Rise time será de",((np.max(V_A_Calc)*1000)/5876),"ns",np.max(V_A_Calc))

else:
    print("Rise time será de",((np.max(V_C_Calc)*1000)/5876),"ns",np.max(V_C_Calc))
    

RESULTADOS PARA PASSAGEM:
Rise time será de 1.8669277165824243 ns 10.970067262638326


In [220]:
end = time.time()
print(end - start)

292.2931830883026


In [221]:
# seed_4stages6_t_7_76=Seed
# %store seed_4stages6_t_7_76